In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, DoubleType, IntegerType
from pyspark.sql.functions import sum as _sum, avg, expr, window, from_unixtime, col

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
import time, datetime
import os
import random
import uuid

spark = (
        SparkSession.builder.appName("spark application")
        .config(
            "spark.jars.packages",
            "org.apache.hudi:hudi-spark3.1-bundle_2.12:0.13.0,"
            "org.apache.hadoop:hadoop-aws:3.1.1,"
            "com.amazonaws:aws-java-sdk:1.11.271,"
        )
        .config(
            "spark.sql.extensions",
            "org.apache.spark.sql.hudi.HoodieSparkSessionExtension",
        )
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
        .config(
            "spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem"
        )
        .config("spark.hadoop.fs.s3a.access.key", "minio")
        .config("spark.hadoop.fs.s3a.secret.key", "minio123")
        .config("spark.hadoop.fs.s3a.endpoint", "http://10.163.16.26:9000")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
        .config(
            "spark.hadoop.fs.s3a.aws.credentials.provider",
            "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
        )
        .config("spark.hadoop.fs.s3a.connection.maximum", "1000")
        .getOrCreate()
    )

23/06/21 10:34:28 WARN Utils: Your hostname, ducdn resolves to a loopback address: 127.0.1.1; using 10.1.124.58 instead (on interface enp1s0)
23/06/21 10:34:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/ducdn/.ivy2/cache
The jars for the packages stored in: /home/ducdn/.ivy2/jars
org.apache.hudi#hudi-spark3.1-bundle_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6f82d8db-b625-434a-97d2-c26136e28d82;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/ducdn/Documents/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.hudi#hudi-spark3.1-bundle_2.12;0.13.0 in central
	found org.apache.hadoop#hadoop-aws;3.1.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.271 in central
	found com.amazonaws#aws-java-sdk;1.11.271 in central
	found com.amazonaws#aws-java-sdk-transcribe;1.11.271 in central
	found com.amazonaws#aws-java-sdk-core;1.11.271 in central
	found commons-logging#commons-logging;1.1.3 in central
	found org.apache.httpcomponents#httpclient;4.5.2 in central
	found org.apache.httpcomponents#httpcore;4.4.4 in central
	found commons-codec#commons-codec;1.9 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found com.fasterxml.jackson.core#jackson-databind;2.6.7.1 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.6.0 in central
	found com.fasterxml.jackson.core#jackson-core;2.6.7 in central
	found com.fasterxml.jackson.dataformat#jackson-dataformat-cbor;2.6.7 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-ja

In [2]:
hudi_path = "s3a://datalake/cdc__test_user"

#### Snapshot or Realtime Queries

In [8]:
df = spark.read.format("hudi").load(hudi_path)
df.show()

+-------------------+--------------------+--------------------+----------------------+--------------------+--------------------+---------------+-------------------+--------------------+----+-----+---+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|      _airbyte_ab_id|_airbyte_stream|_airbyte_emitted_at|       _airbyte_data|year|month|day|
+-------------------+--------------------+--------------------+----------------------+--------------------+--------------------+---------------+-------------------+--------------------+----+-----+---+
|  20230621102446790|20230621102446790...|_airbyte_data.use...|  year=2023/month=6...|792d3a08-9112-437...|845f2dbd-c207-451...|      test_user|      1687311559043|{user3, hungpm3_u...|2023|    6| 21|
|  20230621102446790|20230621102446790...|_airbyte_data.use...|  year=2023/month=6...|792d3a08-9112-437...|4aa91e84-6e06-4bd...|      test_user|      1687311137529|{user5, hungpm5, ...|2023|    6|

In [9]:
df = df.select("_hoodie_commit_time", "_airbyte_data")
df.show(truncate=False)

+-------------------+--------------------------------------------------------------------------------------------------------+
|_hoodie_commit_time|_airbyte_data                                                                                           |
+-------------------+--------------------------------------------------------------------------------------------------------+
|20230621102446790  |{user3, hungpm3_update_3, 2023-06-12T01:59:17.398599Z, 2023-06-21T01:39:10.442Z, 23343864, null}        |
|20230621102446790  |{user5, hungpm5, 2023-06-19T00:03:38.610770Z, 2023-06-21T01:32:18.370Z, 23327296, null}                 |
|20230621102446790  |{user7, hungpm7_insert, 2023-06-21T01:35:04.606959Z, 2023-06-21T01:35:04.610Z, 23327624, null}          |
|20230621102446790  |{user1, hungpm1, 2023-06-12T01:37:57.375694Z, 2023-06-21T01:32:18.370Z, 23327296, null}                 |
|20230621102446790  |{user4, hungpm4, 2023-06-16T01:53:08.860197Z, 2023-06-21T01:32:18.370Z, 23327296, null}   

#### Time travel Queries

In [10]:
df = spark.read.format("hudi").option("as.of.instant","20230621102446790").load(hudi_path)
df.show()

+-------------------+--------------------+--------------------+----------------------+--------------------+--------------------+---------------+-------------------+--------------------+----+-----+---+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|      _airbyte_ab_id|_airbyte_stream|_airbyte_emitted_at|       _airbyte_data|year|month|day|
+-------------------+--------------------+--------------------+----------------------+--------------------+--------------------+---------------+-------------------+--------------------+----+-----+---+
|  20230621102446790|20230621102446790...|_airbyte_data.use...|  year=2023/month=6...|792d3a08-9112-437...|845f2dbd-c207-451...|      test_user|      1687311559043|{user3, hungpm3_u...|2023|    6| 21|
|  20230621102446790|20230621102446790...|_airbyte_data.use...|  year=2023/month=6...|792d3a08-9112-437...|4aa91e84-6e06-4bd...|      test_user|      1687311137529|{user5, hungpm5, ...|2023|    6|

#### Incremental Queries

In [12]:
# Cấu hình truy vấn incremental
incrementalOptions = {
    "hoodie.datasource.query.type": "incremental",
    "hoodie.datasource.read.begin.instanttime": "0",
    # "hoodie.datasource.read.end.instanttime": "20230620100109120"
    # "hoodie.datasource.read.partition.path": "/year=2020/month=*/day=*"  # Tuỳ chọn, sử dụng glob pattern nếu truy vấn các phân vùng cụ thể
}

hudiIncQueryDF = spark.read \
    .format("org.apache.hudi") \
    .options(**incrementalOptions) \
    .load(hudi_path)

hudiIncQueryDF.createOrReplaceTempView("hudi_incre")
spark.sql("select * from hudi_incre order by _hoodie_commit_time").show(truncate=False)

+-------------------+---------------------+---------------------------+------------------------+------------------------------------------------------------------------+------------------------------------+---------------+-------------------+--------------------------------------------------------------------------------------------------------+----+-----+---+
|_hoodie_commit_time|_hoodie_commit_seqno |_hoodie_record_key         |_hoodie_partition_path  |_hoodie_file_name                                                       |_airbyte_ab_id                      |_airbyte_stream|_airbyte_emitted_at|_airbyte_data                                                                                           |year|month|day|
+-------------------+---------------------+---------------------------+------------------------+------------------------------------------------------------------------+------------------------------------+---------------+-------------------+--------------------------------